<a href="https://colab.research.google.com/github/OttoTarkka/kouluta-oma-neuroverkko/blob/main/kouluta_oma_neuroverkko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kouluta oma neuroverkko

Tässä notebookin avulla voit kouluttaa oman luokittelijasi.

Tiedoston Python-koodi on jaettu soluihin, jotka voi ajaa yksi kerrallaan klikkaamalla solun vasemmassa laidassa olevaa "play"-painiketta. Kun teette muokkauksia koodiin, ajakaa sen jälkeen muokattu solu ja kaikki sen jälkeen tulevat solut järjestyksessä, niin koodi toimii oikein.

Python-kielessä kaikki risuaidan (#) jälkeen tuleva on kommenttia. Kommenttiriveillä voidaan antaa tietoa koodin toiminnasta, mutta ne eivät ole osa varsinaista suoritettavaa ohjelmaa.

---

## Tarvittavien pakettien asennus

Pythoniin voi asentaa muiden rakentamia ja julkaisemia paketteja. Nämä paketit yksinkertaistavat monia monimutkaisia tehtäviä, eikä meidän esimerkiksi tarvitse rakentaa koko neuroverkkoarkkitehtuuria alusta alkaen.

Tätä koodia varten tarvitsemme paketit `transformers`, `datasets` ja `evaluate`.

In [ ]:
# Asennetaan paketit komennolla !pip install
!pip install --quiet transformers datasets evaluate

# Asentamisen jälkeen paketit pitää vielä aktivoida käyttöön import-komennolla
import datasets
import transformers
import evaluate

# pprint eli prettyprint tekee tulosteista kivemman näköisiä
from pprint import pprint

# Nämä rivit vähentävät ylimääräistä hälyä
transformers.utils.logging.set_verbosity_error()
datasets.logging.set_verbosity_error()
datasets.disable_progress_bar()

---

## Ladataan koulutusdata

Ladataan valmiiksi laadittu koulutusdata. Käytetään tähän `datasets`-pakettia, joka lataa datan HuggingFace-palvelusta.

In [ ]:
# Vaihtoehtoisia luokitteludatasettejä:
# tunteita: "dair-ai/emotion"
# Lue lisää: https://huggingface.co/datasets/dair-ai/emotion

# elokuva-arvosteluja: "stanfordnlp/imdb"
# Lue lisää: https://huggingface.co/datasets/stanfordnlp/imdb

# spämmiviestejä: "ucirvine/sms_spam"
# Lue lisää: https://huggingface.co/datasets/ucirvine/sms_spam

# uutisaiheita: "fancyzhx/ag_news"
# Lue lisää: https://huggingface.co/datasets/fancyzhx/ag_news

# toksisia viestejä: "mteb/toxic_conversations_50k"
# Lues lisää: https://huggingface.co/datasets/mteb/toxic_conversations_50k

# Tässä voit vaihtaa käytettävää datasettiä!
DATASET = "dair-ai/emotion"

dataset = datasets.load_dataset(DATASET)

# ====================================================================
# Tässä varmistetaan, että datasetin rakenne on sellainen kuin haluamme
# Tästä ei tarvitse välittää!
def validate_data_strucuture(ds: datasets.DatasetDict):
  if not "test" in ds.keys():
    ds = ds["train"].train_test_split(test_size=0.2)
  if not "validation" in ds.keys():
    if "valid" in ds.keys():
      ds = ds.rename_column("valid", "validation")
    elif "dev" in ds.keys():
      ds = ds.rename_column("dev", "validation")
    else:
      ds_devtest = ds["test"].train_test_split(test_size=0.5, seed=42)
      ds = datasets.DatasetDict({
          "train": ds["train"],
          "validation": ds_devtest["train"],
          "test": ds_devtest["test"]
      })

  # Rename 'sms' column to 'text' if it exists
  for split in ds.keys():
      if "sms" in ds[split].column_names:
          ds[split] = ds[split].rename_column("sms", "text")

  max_train_size = 20000
  max_eval_size = 10000
  if len(ds["train"]) > max_train_size:
    ds["train"] = ds["train"].select(range(max_train_size))

  if len(ds["validation"]) > max_eval_size:
    ds["validation"] = ds["validation"].select(range(max_eval_size))

  if len(ds["test"]) > max_eval_size:
    ds["test"] = ds["test"].select(range(max_eval_size))

  if not isinstance(ds["train"].features["label"], datasets.ClassLabel):
    # Check if label_names column exists
    if "label_text" in ds["train"].column_names:
      unique_labels = ds["train"].unique("label_text")
    else:
      # Get unique labels from the training set
      unique_labels = ds["train"].unique("label")
    # Create a ClassLabel object
    class_label = datasets.ClassLabel(
        num_classes=len(unique_labels),
        names=list(sorted(unique_labels))
        )
    # Cast the labels to the ClassLabel type
    for split in ds.keys():
      ds[split] = ds[split].cast_column("label", class_label)

  return ds

dataset = validate_data_strucuture(dataset)
# ====================================================================

Katsotaan, miltä lataamamme datasetti näyttää!

In [ ]:
print(dataset)
print()
print("Yksi esimerkki datasta:")
print(dataset["train"][0])

Datassa tekstin luokat on merkattu numeroilla. Esim 0 = suru,1 = ilo, jne. Muutetaan numerot ihmisluettavaan muotoon.

In [ ]:
label_names = dataset["train"].features["label"].names
print("Luokkien nimet:", label_names)

num_labels = len(label_names)
id2label = { k: v for k, v in enumerate(label_names) }
label2id = { v: k for k, v in enumerate(label_names) }

print("Luokkien määrä:", num_labels)
print("id2label mapping:", id2label)

---

## Valitaan malli ja tokenisoidaan data

Valitaan haluamamme pohjamalli ja ladataan mallille sopiva tokenisoija. Tokenisoidaan data ja muutetaan se numeeriseen muotoon mallin kouluttamista varten.

In [ ]:
# Vaihtoehtosia malleja:
# "google-bert/bert-base-cased"     <- englanninkielinen malli
# "FacebookAI/xlm-roberta-base"     <- monikielinen malli


MODEL = "google-bert/bert-base-cased"
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL)

Katsotaan miltä tokenisoitu teksti näyttää!

In [ ]:
print("Esimerkki tokenisoidusta lauseesta:")
tokenized_example = tokenizer("This is an example sentence")
pprint(tokenized_example["input_ids"])
print()
print()
print("Tokenisoitu lause muutettuna takaisin normaaliksi tekstiksi")
pprint(tokenizer.decode(tokenized_example["input_ids"]))


Tokenisoidaan koko lataamamme datasetti!

In [ ]:
def tokenize(example):
  return tokenizer(example["text"])

dataset = dataset.map(tokenize)

---

## Mallin kouluttaminen

Nyt meidän koulutusdatamme on valmiina. Seuraavaksi ladataan pohjamalli, jota aletaan jatkokouluttaa.

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    MODEL,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

accuracy = evaluate.load("accuracy")

def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels
    predictions = outputs.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

### Säädetään koulutusasetukset kuntoon
Tässä säädetään asetukset kouluttamista varten.
Kokeile muuttaa asetuksia ja katso,
miten se vaikuttaa mallin oppimiseen.
Kuka kouluttaa parhaan mallin?

In [ ]:
# Kokeile muuttaa näitä arvoja ja katso miten se
# vaikuttaa mallin oppimiseen
max_steps = 1000                # kuinka monella esimerkillä mallia koulutetaan
learning_rate = 0.00001         # kuinka nopeasti malli oppii
per_device_train_batch_size = 8 # kuinka monta esimerkkiä malli näkee kerralla


trainer_args = transformers.TrainingArguments(
  output_dir="checkpoints",
  eval_strategy="steps",
  logging_strategy="steps",
  load_best_model_at_end=True,
  eval_steps=100,
  logging_steps=100,
  per_device_eval_batch_size=32,
  save_strategy="steps",
  save_steps=1000,
  max_steps=max_steps,
  learning_rate=learning_rate,
  per_device_train_batch_size=per_device_train_batch_size
)

---

## Viimeinkin päästää kouluttamaan!

In [ ]:
trainer = transformers.Trainer(
    model=model,
    args=trainer_args,
    train_dataset=dataset["train"], # treenataan "train" datalla
    eval_dataset=dataset["validation"], # evaluoidaan "validation" datalla
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
)

trainer.train()

---

## Testataan, kuinka hyvin mallimme toimii!

Tässä käytetään testidataa, jota malli ei ole nähnyt lainkaan kouluttamisen aikana.

In [ ]:
eval_results = trainer.evaluate(dataset["test"])
print()
print("Mallin tarkkuus testidatalla:", round(eval_results["eval_accuracy"]*100, 2), "%")

---

## Kokeille itse, miten malli toimii!

Keksi lauseita ja katso, minkä luokan malli niille ennustaa!
Muista, että jos antamasi lauseet ovat hyvin erilaisia kuin koulutusdata, malli ei todennäköisesti toimi kovin hyvin.



In [ ]:
# Valmistellaan "pipeline", joka tokenisoi annetun tekstin
# ja ajaa sen mallin läpi automaattisesti
pipe = transformers.pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device=0
)

In [ ]:
# Kirjoita oma lauseesi tähän!
oma_lause = "This model is great!"

pred = pipe(oma_lause)
print("Ennustettu luokka:", pred[0]["label"])
print("Ennusteen varmuus:", round(pred[0]["score"]*100,2),"%")